In [1]:
import os
import openai
from interp_utils import reload_module
reload_module('noa_tools')
from noa_tools import threaded_query, query
from tokenizers import Tokenizer

import torch
from openai import OpenAI

client = OpenAI()


top_10k = torch.load('top_10k_indices.pt')
tokenizer = Tokenizer.from_pretrained('roneneldan/TinyStories-33M')


In [2]:
toks = [tokenizer.decode([tok_id]) for tok_id in top_10k]
toks = list(set([tok.lower()[1:] for tok in toks if tok[0] == ' ']))
# toks = [tok for tok in toks if tok[0] == ' ']

In [3]:


# prompt = '''Given a list of strings, identify which are single things by calling log_single_things with strings are singular instead of plural and are actual words. For example, 'fur', 'mountain', 'wagon', 'rock', 'candle' could all be single objects, but 'furs', 'mountains', 'wagons', 'rocks', 'candles' are not. Copy strings exactly as they appear.

# Here is the list of strings: {batch}'''

# functions = [
#     {
#     "name": "log_single_things",
#     "description": "Logs all identified single things.",
#     "parameters": {
#         "type": "object",
#         "properties": {
#             "single_things": {
#                 "type": "array",
#                 "description": "A list of identified single things",
#                 "items": {
#                     "type": "string",
#                 }
#             },
#         },
#     },
#     "required": ["single_things"],
#     },
# ]

In [4]:

LOG_FN_STR = 'water_words'
log_fn = 'log_'+LOG_FN_STR
# strings that are {DESCRIPTION}
DESCRIPTION = 'water-related'
EXAMPLES=['water', 'bath', 'overflow', 'bathroom', 'tank', 'wet', 'ducks']

example_str = ', '.join(["'"+word+"'" for word in EXAMPLES])

prompt = f'''Given a list of strings, identify which are related to water by calling LOG_FN_NAMEwith strings that are {DESCRIPTION}. Copy strings exactly as they appear. For example, {example_str}.

Here is the list of strings: {{batch}}'''

functions = [
    {
    "name": log_fn,
    "description": "",
    "parameters": {
        "type": "object",
        "properties": {
            LOG_FN_STR: {
                "type": "array",
                "description": "",
                "items": {
                    "type": "string",
                }
            },
        },
    },
    "required": [log_fn],
    },
]


In [5]:

from noa_tools import threaded_map
import itertools
import json

def get_words_from_batch(batch):
    out = query(prompt=prompt.format(batch=batch), messages=[], model='gpt-4', functions=functions, function_call={'name': log_fn}, temperature=0.9)

    try:
        outputted_words = json.loads(
            out.choices[0].message.function_call.arguments
        )[LOG_FN_STR]
    except Exception as e:
        print(f'Exception: {e}')
        return []

    words = []
    for candidate_token in outputted_words:
        if candidate_token not in batch:
            continue
        for tok in [' '+candidate_token, candidate_token.capitalize(), ' '+candidate_token.capitalize(), candidate_token]:
            ids = tokenizer.encode(tok).ids
            if len(ids) != 1:
                continue
            words.append(tok)

    return words



BATCH_SIZE = 70

counts = torch.zeros(50258).to(torch.int)

from tqdm import tqdm
import random

# inanimate_objects = json.load(open('inanimate_objects.json'))
for _ in tqdm(range(3)):
    random.shuffle(toks)
    batches = [toks[i:i+BATCH_SIZE] for i in range(0, len(toks), BATCH_SIZE)]

    out = list(itertools.chain.from_iterable(threaded_map(get_words_from_batch, batches, n_threads=20)))

    for word in out:
        tok_id = tokenizer.encode(word).ids[0]
        counts[tok_id] += 1



100%|██████████| 3/3 [00:22<00:00,  7.45s/it]


In [6]:
identified_words = (counts >= 2).int().nonzero().flatten()
identified_words = [tokenizer.decode([word]) for word in identified_words]
identified_words = list(set(identified_words))
torch.save(identified_words, LOG_FN_STR+'.pt')

In [19]:
past_tense_verbs = [tokenizer.decode([tok_id]) for tok_id in (counts >= 2).int().nonzero().flatten()]
# out = [tok for tok in out if tok[0] == ' ' and tok[1].islower()]
json.dump(past_tense_verbs, open('past_tense_verbs.json', 'w'))


: 

In [37]:
# batches[0]

In [65]:
turbo = (counts >= 2).int().nonzero().flatten()
turbo = [tokenizer.decode([adj]) for adj in turbo]
inanimate_objects = [word for word in turbo if word[0] == ' ' or word.capitalize() == word]
# turbo = [word for word in turbo if word[0] == ' ' or word.capitalize() == word]
# turbo
# json.dump(turbo, open('inanimate_objects.json', 'w'))
inanimate_objects

[]

In [232]:
# adjectives = [tokenizer.decode([adj]) for adj in turbo]
# adjectives
# torch.save(turbo, 'turbo-intensifiers.pt')
adjectives = [adj for adj in torch.load('turbo-adjectives.pt') if adj[0] == ' ']
adjectives

[' other',
 ' new',
 ' only',
 ' first',
 ' two',
 ' most',
 ' even',
 ' every',
 ' right',
 ' very',
 ' many',
 ' well',
 ' long',
 ' good',
 ' last',
 ' New',
 ' same',
 ' high',
 ' great',
 ' real',
 ' really',
 ' three',
 ' public',
 ' few',
 ' different',
 ' second',
 ' fun',
 ' big',
 ' best',
 ' open',
 ' far',
 ' next',
 ' little',
 ' hard',
 ' full',
 ' less',
 ' top',
 ' better',
 ' small',
 ' current',
 ' old',
 ' near',
 ' free',
 ' able',
 ' enough',
 ' large',
 ' important',
 ' clear',
 ' kind',
 ' mean',
 ' sure',
 ' light',
 ' human',
 ' available',
 ' certain',
 ' official',
 ' possible',
 ' proper',
 ' short',
 ' several',
 ' complete',
 ' young',
 ' low',
 ' early',
 ' based',
 ' strong',
 ' five',
 ' present',
 ' particular',
 ' local',
 ' former',
 ' close',
 ' special',
 ' black',
 ' single',
 ' half',
 ' quick',
 ' true',
 ' bad',
 ' similar',
 ' entire',
 ' previous',
 ' whole',
 ' super',
 ' common',
 ' North',
 ' red',
 ' Red',
 ' white',
 ' third',
 ' longer'

In [180]:
'tree'.capitalize()

'Tree'

In [210]:
# turbo = torch.load('turbo-adjectives.pt')
normal = torch.load('adjectives.pt')

In [218]:
set(normal)-set(turbo)


{' Another',
 ' Being',
 ' Everywhere',
 ' Flash',
 ' Gold',
 ' Love',
 ' Luckily',
 ' More',
 ' Much',
 ' Often',
 ' Outside',
 ' Peach',
 ' Power',
 ' Seeing',
 ' Slowly',
 ' Sorry',
 ' Spanish',
 ' Speed',
 ' Still',
 ' Summer',
 ' Sunshine',
 ' Tree',
 ' Unfortunately',
 ' Winter',
 ' Zero',
 ' above',
 ' accepted',
 ' actually',
 ' afloat',
 ' almost',
 ' along',
 ' already',
 ' alright',
 ' also',
 ' another',
 ' any',
 ' appropriately',
 ' attention',
 ' autumn',
 ' awake',
 ' awe',
 ' backyard',
 ' badly',
 ' baking',
 ' barely',
 ' beneath',
 ' bleeding',
 ' blown',
 ' blues',
 ' boiled',
 ' boldly',
 ' both',
 ' breeze',
 ' brilliantly',
 ' bunny',
 ' buzzing',
 ' calmly',
 ' caramel',
 ' caring',
 ' celebrated',
 ' certainly',
 ' change',
 ' changing',
 ' cherry',
 ' chocolate',
 ' cinnamon',
 ' clearly',
 ' closed',
 ' closer',
 ' coconut',
 ' colour',
 ' colours',
 ' combined',
 ' comfortably',
 ' complaining',
 ' completed',
 ' controlling',
 ' cooled',
 ' correctly',
 ' 

In [123]:
tokenizer.decode(list(counts.nonzero().flatten()))

' so Last healthy stupid welcomed boiled piercing'